In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from collections import Counter

# Cargar los datos
archivo_csv = "df_limpio.csv"
df = pd.read_csv(archivo_csv)

# Seleccionar las columnas relevantes
columnas_seleccionadas = ['Horas_Estudio', 'Asistencia', 'Horas_Sueño', 
                          'Puntajes_Anteriores', 'Sesiones_Tutorias', 'Actividad_Fisica']
X = df[columnas_seleccionadas]
y = df['Puntaje_Examen']

# Discretizar 'y' en Aprobado/Reprobado
y = y.apply(lambda x: 1 if x >= 60 else 0)

# Manejar valores faltantes
X = X.apply(pd.to_numeric, errors='coerce')
X = X.fillna(X.mean())  # Imputar valores faltantes con la media

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Balancear las clases con SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Verificar el balance de clases después de SMOTE
print("Distribución después de SMOTE:", Counter(y_train_smote))


Distribución después de SMOTE: Counter({1: 5291, 0: 5291})


In [3]:
# Escalar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_smote)
X_test_scaled = scaler.transform(X_test)

# Entrenar el modelo de Regresión Logística
modelo_lr = LogisticRegression(max_iter=1000, class_weight='balanced')
modelo_lr.fit(X_train_scaled, y_train_smote)

# Hacer predicciones
y_pred_lr = modelo_lr.predict(X_test_scaled)

# Evaluar el modelo
print("Resultados con Regresión Logística:")
print(f'Precisión: {accuracy_score(y_test, y_pred_lr)}')
print("Reporte de clasificación:")
print(classification_report(y_test, y_pred_lr))
print("Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred_lr))


Resultados con Regresión Logística:
Precisión: 0.6024096385542169
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.06      0.46      0.11        79
           1       0.95      0.61      0.74      1332

    accuracy                           0.60      1411
   macro avg       0.51      0.53      0.43      1411
weighted avg       0.90      0.60      0.71      1411

Matriz de Confusión:
[[ 36  43]
 [518 814]]


In [4]:
# Entrenar el modelo de Random Forest
modelo_rf = RandomForestClassifier(random_state=42, class_weight='balanced')
modelo_rf.fit(X_train_smote, y_train_smote)

# Hacer predicciones
y_pred_rf = modelo_rf.predict(X_test)

# Evaluar el modelo
print("\nResultados con Random Forest:")
print(f'Precisión: {accuracy_score(y_test, y_pred_rf)}')
print("Reporte de clasificación:")
print(classification_report(y_test, y_pred_rf))
print("Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred_rf))



Resultados con Random Forest:
Precisión: 0.9390503189227498
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.27      0.05      0.09        79
           1       0.95      0.99      0.97      1332

    accuracy                           0.94      1411
   macro avg       0.61      0.52      0.53      1411
weighted avg       0.91      0.94      0.92      1411

Matriz de Confusión:
[[   4   75]
 [  11 1321]]


In [5]:
from sklearn.model_selection import GridSearchCV

# Definir el grid de hiperparámetros
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

# Configurar GridSearchCV
grid_search = GridSearchCV(RandomForestClassifier(random_state=42, class_weight='balanced'),
                           param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=2)

# Entrenar el modelo con GridSearch
grid_search.fit(X_train_smote, y_train_smote)

# Imprimir los mejores parámetros y evaluar
print("\nMejores parámetros:", grid_search.best_params_)
mejor_modelo = grid_search.best_estimator_

# Evaluar el modelo optimizado
y_pred_optimizado = mejor_modelo.predict(X_test)
print("\nResultados con el Modelo Optimizado:")
print(f'Precisión: {accuracy_score(y_test, y_pred_optimizado)}')
print("Reporte de clasificación:")
print(classification_report(y_test, y_pred_optimizado))
print("Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred_optimizado))


Fitting 3 folds for each of 81 candidates, totalling 243 fits

Mejores parámetros: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}

Resultados con el Modelo Optimizado:
Precisión: 0.9383416017009213
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.21      0.04      0.06        79
           1       0.95      0.99      0.97      1332

    accuracy                           0.94      1411
   macro avg       0.58      0.51      0.52      1411
weighted avg       0.90      0.94      0.92      1411

Matriz de Confusión:
[[   3   76]
 [  11 1321]]
